In [78]:
# !apt install chromium-chromedriver -y

In [79]:
import pandas as pd
import joblib
import tqdm
from pathlib import Path
from omegaconf import OmegaConf

In [80]:
cfg = OmegaConf.load('conf/config.yaml')

for k, path in cfg.catalog.items():
    Path(path).parent.mkdir(parents=True, exist_ok=True)
train = pd.read_csv(cfg.catalog.train)
test = pd.read_csv(cfg.catalog.test)

cat = pd.concat(
    [
     train.assign(split='train', idx=lambda x: x.index),
     test.assign(split='test', idx = lambda x: x.index),
    ],     ignore_index=True)
cat
cat['en_title'] = ''
cat['en_cleanBody'] = '' 
cat['done'] = False 
cat['global_idx'] = cat.index.copy()



In [81]:
table = cat.query('split=="train"').groupby('category').sample(1)
table

,title,cleanBody,category,split,idx,en_title,en_cleanBody,done,global_idx
1874,시크한 윤여정이 흥분했다…美배우조합상 여우조연상 韓최초 수상(2보),배우 윤여정(74)이 미국배우조합상 시상식에서 여우조연상을 수상했다. 한국 배우 최...,culture,train,1874,,,False,1874
92,SK바이오사이언스가 주목받는 3가지 이유,"""수익성, 성장성, 전문성 등 초대형 바이오기업이 갖춰야 할 필수적인 3박자를 모두...",economy,train,92,,,False,92
277,"'320kg 활동중단' 빅죠, 충격 근황 ""목에 튜브 연결해 호흡…팔·다리 묶기도""",가수 빅죠의 근황이 많은 이들에게 충격을 안겼다. 유튜브 채널 엄상용은 지난 27일...,entertain,train,277,,,False,277
2042,"日, 후쿠시마 오염수 방류 강행…제주 앞바다 오염까지 200일",도쿄전력 후쿠시마 제1원전의 오염수 보관 탱크. 일본 정부가 후쿠시마 제1원자력발전...,international,train,2042,,,False,2042
165,"신세계 손잡은 네이버, 이베이 노리는 카카오…'목표는 타도쿠팡'",IT업계 양대 산맥인 네이버(NAVER)와 카카오가 이커머스 분야에서 몸집 불리기에...,it,train,165,,,False,165
1626,'주4일제 도입' 우상호-조정훈 '맞장 토론',시장 선거에 나선 우상호 더불어민주당 의원이 범여권 경쟁자인 조정훈 시대전환 의원을...,politics,train,1626,,,False,1626
1229,"이용구 사무실서 '尹감찰' 조사 논란…""전혀 몰랐다"" 해명",법무부 감찰관실이 윤석열 검찰총장을 둘러싼 의혹과 관련해 박상기 전 법무부 장관을 ...,society,train,1229,,,False,1229
1710,"한일전서 이동준 가격에 치아 빠진 日선수 ""큰 문제되지 않길""",지난 25일 한일 축구 국가 대표팀 친선전에서 이동준의 팔꿈치 가격으로 치아가 빠진...,sport,train,1710,,,False,1710


In [83]:
# table = joblib.load(cfg.catalog.en)

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time

language_code = "te"
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get('https://translate.google.com/#view=home&op=translate&sl=ko&tl=en')

trt_col = 'title'
for gidx, src_sentence in tqdm.tqdm(
    table[['global_idx',trt_col]].itertuples(index=False)
):
    
    driver.find_element_by_xpath("//textarea[@aria-label='Source text']").clear() 
    time.sleep(3)
    driver.find_element_by_xpath("//textarea[@aria-label='Source text']").send_keys(src_sentence) 
    time.sleep(3)
    spans = driver.find_elements_by_xpath("//span[@lang='en']/span")
    trt_sentence = ''.join([span.text for span in spans])
    table.loc[table.global_idx == gidx, trt_col] = trt_sentence


/tmp/ipykernel_388126/214977863.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
0it [00:00, ?it/s]/tmp/ipykernel_388126/214977863.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//textarea[@aria-label='Source text']").clear()
/tmp/ipykernel_388126/214977863.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//textarea[@aria-label='Source text']").send_keys(src_sentence)
/tmp/ipykernel_388126/214977863.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  spans = driver.find_elements_by_xpath("//span[@lang='en']/span")
8it [00:50,  6.36s/it]


In [84]:
table

,title,cleanBody,category,split,idx,en_title,en_cleanBody,done,global_idx
1874,Chic Yoon Yeo -jung was excited…American Actre...,배우 윤여정(74)이 미국배우조합상 시상식에서 여우조연상을 수상했다. 한국 배우 최...,culture,train,1874,,,False,1874
92,Three reasons for SK Bioscience,"""수익성, 성장성, 전문성 등 초대형 바이오기업이 갖춰야 할 필수적인 3박자를 모두...",economy,train,92,,,False,92
277,"'320kg stop' Big Jo, Shock, ""Tube connects to ...",가수 빅죠의 근황이 많은 이들에게 충격을 안겼다. 유튜브 채널 엄상용은 지난 27일...,entertain,train,277,,,False,277
2042,"Japan, Fukushima Pollution Discharge…200 days ...",도쿄전력 후쿠시마 제1원전의 오염수 보관 탱크. 일본 정부가 후쿠시마 제1원자력발전...,international,train,2042,,,False,2042
165,"Shinsegae Handing Naver, eBay Nori Kakao…'The ...",IT업계 양대 산맥인 네이버(NAVER)와 카카오가 이커머스 분야에서 몸집 불리기에...,it,train,165,,,False,165
1626,'The 4th Date introduction' Woo Sang-ho-Cho Ju...,시장 선거에 나선 우상호 더불어민주당 의원이 범여권 경쟁자인 조정훈 시대전환 의원을...,politics,train,1626,,,False,1626
1229,Controversy over the investigation of the 'ins...,법무부 감찰관실이 윤석열 검찰총장을 둘러싼 의혹과 관련해 박상기 전 법무부 장관을 ...,society,train,1229,,,False,1229
1710,Japan's player who was missing at the price of...,지난 25일 한일 축구 국가 대표팀 친선전에서 이동준의 팔꿈치 가격으로 치아가 빠진...,sport,train,1710,,,False,1710
